In [40]:
##### file path
### input
# data_set keys and lebels
path_df_part_1_uic_label = "mobile/df_part_1_uic_label.csv"
path_df_part_2_uic_label = "mobile/df_part_2_uic_label.csv"
path_df_part_3_uic       = "mobile/df_part_3_uic.csv"

# data_set features
path_df_part_1_U   = "mobile/feature/df_part_1_U.csv"  
path_df_part_1_I   = "mobile/feature/df_part_1_I.csv"
path_df_part_1_C   = "mobile/feature/df_part_1_C.csv"
path_df_part_1_IC  = "mobile/feature/df_part_1_IC.csv"
path_df_part_1_UI  = "mobile/feature/df_part_1_UI.csv"
path_df_part_1_UC  = "mobile/feature/df_part_1_UC.csv"

path_df_part_2_U   = "mobile/feature/df_part_2_U.csv"  
path_df_part_2_I   = "feature/df_part_2_I.csv"
path_df_part_2_C   = "feature/df_part_2_C.csv"
path_df_part_2_IC  = "feature/df_part_2_IC.csv"
path_df_part_2_UI  = "feature/df_part_2_UI.csv"
path_df_part_2_UC  = "feature/df_part_2_UC.csv"

path_df_part_3_U   = "feature/df_part_3_U.csv"  
path_df_part_3_I   = "feature/df_part_3_I.csv"
path_df_part_3_C   = "feature/df_part_3_C.csv"
path_df_part_3_IC  = "feature/df_part_3_IC.csv"
path_df_part_3_UI  = "feature/df_part_3_UI.csv"
path_df_part_3_UC  = "feature/df_part_3_UC.csv"

### out file

### intermediate file
# data partition with diffferent label
path_df_part_1_uic_label_0 = "mobile/gbdt/k_means_subsample/df_part_1_uic_label_0.csv"
path_df_part_1_uic_label_1 = "mobile/gbdt/k_means_subsample/df_part_1_uic_label_1.csv"

# training set keys uic-label with k_means clusters' label
path_df_part_1_uic_label_cluster = "mobile/gbdt/k_means_subsample/df_part_1_uic_label_cluster.csv"

# for each part's features
path_df_part_1_scaler = "mobile/gbdt/k_means_subsample/df_part_1_scaler"


In [2]:
import pandas as pd
import numpy as np
def df_read(path, mode = 'r'):
    '''the definition of dataframe loading function 
    '''
    path_df = open(path, mode)
    df = pd.read_csv(path_df, index_col = False)
    return   df

def subsample(df, sub_size):
    '''the definition of sub-sampling function
    @param df: dataframe
    @param sub_size: sub_sample set size
    
    @return sub-dataframe with the same formation of df
    '''
    if sub_size >= len(df) : 
        return df
    else : 
        return df.sample(n = sub_size)  


In [7]:
########################################################################
'''Step 1: dividing of positive and negative sub-set by u-i-c-label keys
    
    p.s. we first generate u-i-C key, then merging for data set and operation by chunk 
    such strange operation designed for saving my poor PC-MEM.
'''

df_part_1_uic_label = df_read(path_df_part_1_uic_label)
df_part_1_uic_label_0 = df_part_1_uic_label[df_part_1_uic_label['label'] == 0]
df_part_1_uic_label_1 = df_part_1_uic_label[df_part_1_uic_label['label'] == 1]
df_part_1_uic_label_0.to_csv(path_df_part_1_uic_label_0, index=False)
df_part_1_uic_label_1.to_csv(path_df_part_1_uic_label_1, index=False)



In [8]:
#######################################################################
'''Step 2: clustering on negative sub-set
    clusters number ~ 35, using mini-batch-k-means
'''

# clustering based on sklearn
from sklearn import preprocessing
from sklearn.cluster import MiniBatchKMeans
import pickle

##### part_1 #####
# loading features
df_part_1_U  = df_read(path_df_part_1_U )   
df_part_1_I  = df_read(path_df_part_1_I )
df_part_1_C  = df_read(path_df_part_1_C )
df_part_1_IC = df_read(path_df_part_1_IC)
df_part_1_UI = df_read(path_df_part_1_UI)
df_part_1_UC = df_read(path_df_part_1_UC)

scaler_1 = preprocessing.StandardScaler() 
batch = 0
for df_part_1_uic_label_0 in pd.read_csv(open(path_df_part_1_uic_label_0, 'r'), chunksize=150000): 
    # construct of part_1's sub-training set
    train_data_df_part_1 = pd.merge(df_part_1_uic_label_0, df_part_1_U, how='left', on=['user_id'])
    train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_I,  how='left', on=['item_id'])
    train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_C,  how='left', on=['item_category'])
    train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_IC, how='left', on=['item_id','item_category'])
    train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_UI, how='left', on=['user_id','item_id','item_category','label'])
    train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_UC, how='left', on=['user_id','item_category'])
    
    # getting all the complete features for clustering
    train_X_1 = train_data_df_part_1.as_matrix(['u_b1_count_in_6','u_b2_count_in_6','u_b3_count_in_6','u_b4_count_in_6','u_b_count_in_6', 
                                                    'u_b1_count_in_3','u_b2_count_in_3','u_b3_count_in_3','u_b4_count_in_3','u_b_count_in_3', 
                                                    'u_b1_count_in_1','u_b2_count_in_1','u_b3_count_in_1','u_b4_count_in_1','u_b_count_in_1', 
                                                    'u_b4_rate',
                                                    'i_u_count_in_6','i_u_count_in_3','i_u_count_in_1',
                                                    'i_b1_count_in_6','i_b2_count_in_6','i_b3_count_in_6','i_b4_count_in_6','i_b_count_in_6', 
                                                    'i_b1_count_in_3','i_b2_count_in_3','i_b3_count_in_3','i_b4_count_in_3','i_b_count_in_3',
                                                    'i_b1_count_in_1','i_b2_count_in_1','i_b3_count_in_1','i_b4_count_in_1','i_b_count_in_1', 
                                                    'i_b4_rate',
                                                    'c_b1_count_in_6','c_b2_count_in_6','c_b3_count_in_6','c_b4_count_in_6','c_b_count_in_6',
                                                    'c_b1_count_in_3','c_b2_count_in_3','c_b3_count_in_3','c_b4_count_in_3','c_b_count_in_3',
                                                    'c_b1_count_in_1','c_b2_count_in_1','c_b3_count_in_1','c_b4_count_in_1','c_b_count_in_1',
                                                    'c_b4_rate',
                                                    'ic_u_rank_in_c','ic_b_rank_in_c','ic_b4_rank_in_c', 
                                                    'ui_b1_count_in_6','ui_b2_count_in_6','ui_b3_count_in_6','ui_b4_count_in_6','ui_b_count_in_6',
                                                    'ui_b1_count_in_3','ui_b2_count_in_3','ui_b3_count_in_3','ui_b4_count_in_3','ui_b_count_in_3',
                                                    'ui_b1_count_in_1','ui_b2_count_in_1','ui_b3_count_in_1','ui_b4_count_in_1','ui_b_count_in_1', 
                                                    'ui_b_count_rank_in_u','ui_b_count_rank_in_uc',
                                                    'uc_b1_count_in_6','uc_b2_count_in_6','uc_b3_count_in_6','uc_b4_count_in_6','uc_b_count_in_6', 
                                                    'uc_b1_count_in_3','uc_b2_count_in_3','uc_b3_count_in_3','uc_b4_count_in_3','uc_b_count_in_3', 
                                                    'uc_b1_count_in_1','uc_b2_count_in_1','uc_b3_count_in_1','uc_b4_count_in_1','uc_b_count_in_1',
                                                    'uc_b_count_rank_in_u'])
    # feature standardization
    scaler_1.partial_fit(train_X_1)        
        
    batch += 1
    print('chunk %d done.' %batch) 




/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:53: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


chunk 1 done.
chunk 2 done.
chunk 3 done.
chunk 4 done.
chunk 5 done.
chunk 6 done.
chunk 7 done.
chunk 8 done.
chunk 9 done.
chunk 10 done.
chunk 11 done.
chunk 12 done.


In [27]:
# initial clusters
mbk_1 = MiniBatchKMeans(init='k-means++', n_clusters=1000, batch_size=500, reassignment_ratio=10**-4) 
classes_1 = []
batch = 0
for df_part_1_uic_label_0 in pd.read_csv(open(path_df_part_1_uic_label_0, 'r'), chunksize=15000): 
 
        # construct of part_1's sub-training set
        train_data_df_part_1 = pd.merge(df_part_1_uic_label_0, df_part_1_U, how='left', on=['user_id'])
        train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_I,  how='left', on=['item_id'])
        train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_C,  how='left', on=['item_category'])
        train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_IC, how='left', on=['item_id','item_category'])
        train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_UI, how='left', on=['user_id','item_id','item_category','label'])
        train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_UC, how='left', on=['user_id','item_category'])
        
        train_X_1 = train_data_df_part_1.as_matrix(['u_b1_count_in_6','u_b2_count_in_6','u_b3_count_in_6','u_b4_count_in_6','u_b_count_in_6', 
                                                    'u_b1_count_in_3','u_b2_count_in_3','u_b3_count_in_3','u_b4_count_in_3','u_b_count_in_3', 
                                                    'u_b1_count_in_1','u_b2_count_in_1','u_b3_count_in_1','u_b4_count_in_1','u_b_count_in_1', 
                                                    'u_b4_rate',
                                                    'i_u_count_in_6','i_u_count_in_3','i_u_count_in_1',
                                                    'i_b1_count_in_6','i_b2_count_in_6','i_b3_count_in_6','i_b4_count_in_6','i_b_count_in_6', 
                                                    'i_b1_count_in_3','i_b2_count_in_3','i_b3_count_in_3','i_b4_count_in_3','i_b_count_in_3',
                                                    'i_b1_count_in_1','i_b2_count_in_1','i_b3_count_in_1','i_b4_count_in_1','i_b_count_in_1', 
                                                    'i_b4_rate',
                                                    'c_b1_count_in_6','c_b2_count_in_6','c_b3_count_in_6','c_b4_count_in_6','c_b_count_in_6',
                                                    'c_b1_count_in_3','c_b2_count_in_3','c_b3_count_in_3','c_b4_count_in_3','c_b_count_in_3',
                                                    'c_b1_count_in_1','c_b2_count_in_1','c_b3_count_in_1','c_b4_count_in_1','c_b_count_in_1',
                                                    'c_b4_rate',
                                                    'ic_u_rank_in_c','ic_b_rank_in_c','ic_b4_rank_in_c', 
                                                    'ui_b1_count_in_6','ui_b2_count_in_6','ui_b3_count_in_6','ui_b4_count_in_6','ui_b_count_in_6',
                                                    'ui_b1_count_in_3','ui_b2_count_in_3','ui_b3_count_in_3','ui_b4_count_in_3','ui_b_count_in_3',
                                                    'ui_b1_count_in_1','ui_b2_count_in_1','ui_b3_count_in_1','ui_b4_count_in_1','ui_b_count_in_1', 
                                                    'ui_b_count_rank_in_u','ui_b_count_rank_in_uc',
                                                    'uc_b1_count_in_6','uc_b2_count_in_6','uc_b3_count_in_6','uc_b4_count_in_6','uc_b_count_in_6', 
                                                    'uc_b1_count_in_3','uc_b2_count_in_3','uc_b3_count_in_3','uc_b4_count_in_3','uc_b_count_in_3', 
                                                    'uc_b1_count_in_1','uc_b2_count_in_1','uc_b3_count_in_1','uc_b4_count_in_1','uc_b_count_in_1',
                                                    'uc_b_count_rank_in_u'])
        # feature standardization
        standardized_train_X_1 = scaler_1.transform(train_X_1)
         
        # fit clustering model
        mbk_1.partial_fit(standardized_train_X_1)
    
        print(len(np.unique(mbk_1.labels_)))
        
     
        classes_1 = np.append(classes_1, mbk_1.labels_)
        
        batch += 1
        print('chunk %d done.' %batch) 


/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:36: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


1000
chunk 1 done.
709
chunk 2 done.
693
chunk 3 done.
735
chunk 4 done.
702
chunk 5 done.
696
chunk 6 done.
701
chunk 7 done.
729
chunk 8 done.
688
chunk 9 done.
678
chunk 10 done.
725
chunk 11 done.
690
chunk 12 done.
721
chunk 13 done.
668
chunk 14 done.
696
chunk 15 done.
731
chunk 16 done.
715
chunk 17 done.
729
chunk 18 done.
688
chunk 19 done.
704
chunk 20 done.
718
chunk 21 done.
656
chunk 22 done.
717
chunk 23 done.
737
chunk 24 done.
718
chunk 25 done.
714
chunk 26 done.
704
chunk 27 done.
712
chunk 28 done.
748
chunk 29 done.
710
chunk 30 done.
717
chunk 31 done.
725
chunk 32 done.
708
chunk 33 done.
705
chunk 34 done.
688
chunk 35 done.
699
chunk 36 done.
741
chunk 37 done.
720
chunk 38 done.
744
chunk 39 done.
706
chunk 40 done.
695
chunk 41 done.
709
chunk 42 done.
734
chunk 43 done.
707
chunk 44 done.
707
chunk 45 done.
714
chunk 46 done.
730
chunk 47 done.
720
chunk 48 done.
707
chunk 49 done.
718
chunk 50 done.
708
chunk 51 done.
707
chunk 52 done.
701
chunk 53 done.
7

In [30]:
pickle.dump(scaler_1, open(path_df_part_1_scaler,'wb')) 

In [31]:
classes_1

array([694., 493., 488., ..., 352., 243., 555.])

In [32]:
classes_1.astype('int') + 1

array([695, 494, 489, ..., 353, 244, 556])

In [33]:
classes_1.shape

(1707538,)

In [34]:
# add a new attr for keys
df_part_1_uic_label_0 = df_read(path_df_part_1_uic_label_0)
df_part_1_uic_label_1 = df_read(path_df_part_1_uic_label_1)
df_part_1_uic_label_0.head()

,user_id,item_id,item_category,label
0,10001082,262112219,9614,0
1,10001082,265860266,9614,0
2,100029775,214252945,10392,0
3,100029775,118033464,4120,0
4,100029775,365505232,10223,0


In [35]:
df_part_1_uic_label_0.shape

(1707538, 4)

In [36]:
df_part_1_uic_label_0['class'] = classes_1.astype('int') + 1
df_part_1_uic_label_1['class'] = 0
df_part_1_uic_label_0.head()

,user_id,item_id,item_category,label,class
0,10001082,262112219,9614,0,695
1,10001082,265860266,9614,0,494
2,100029775,214252945,10392,0,489
3,100029775,118033464,4120,0,322
4,100029775,365505232,10223,0,241


In [38]:
df_part_1_uic_label_1.head()

,user_id,item_id,item_category,label,class
0,100295908,6768726,5271,1,0
1,100764607,269976923,3368,1,0
2,100764607,313891623,3368,1,0
3,101982247,248937771,3673,1,0
4,101982247,269964023,2993,1,0


In [41]:
# add a new attr for keys
df_part_1_uic_label_0 = df_read(path_df_part_1_uic_label_0)
df_part_1_uic_label_1 = df_read(path_df_part_1_uic_label_1)

df_part_1_uic_label_0['class'] = classes_1.astype('int') + 1
df_part_1_uic_label_1['class'] = 0

df_part_1_uic_label_class = pd.concat([df_part_1_uic_label_0, df_part_1_uic_label_1])
df_part_1_uic_label_class.to_csv(path_df_part_1_uic_label_cluster, index=False)

In [43]:
df_part_1_uic_label_class

,user_id,item_id,item_category,label,class
0,10001082,262112219,9614,0,695
1,10001082,265860266,9614,0,494
2,100029775,214252945,10392,0,489
3,100029775,118033464,4120,0,322
4,100029775,365505232,10223,0,241
5,100068031,174737964,10120,0,29
6,100068031,354519206,7978,0,29
7,100068031,80207620,9933,0,29
8,100068031,388230080,11406,0,365
9,100068031,318618469,4588,0,365
